In [1]:
import random
import copy
import os
import time
import math
import csv

try:
    from tkinter import *
    from tkinter.ttk import *
except Exception as e:
    print("[ERROR]: {0}".format(e))
    from Tkinter import *

lista_de_cidades =[]

# probabilidade de uma rota sofrer mutação
k_mut_prob = 0.25

# Número de gerações que devem ser geradas
k_n_generations = 100

# Tamanho de cada população
k_population_size = 1000

# Size of the tournament selection. 
tournament_size = 13

# se meritocraciao for True então o melhor the cada geração vai ser levado em diante
meritocracia = True

# Booleano pra ler de um excel
csv_cities = False

# nome do arquivo excel
csv_name = 'cities.csv'


# City class
class City(object):
    """
    Guarda cidades. Na inicialização, automaticamente faz append com lista_de_cidades

    self.x: x-coord
    self.y: y-coord
    self.graph_x: x-coord para representação gráfica
    self.graph_y: y-coord para representação gráfica
    self.name: nome para humanos
    self.distance_to: dicionário de distância para outras cidades

    """
    def __init__(self, name, x, y, distance_to=None):
        # nomes e coordenadas:
        self.name = name
        self.x = self.graph_x = x
        self.y = self.graph_y = y
        # append ao lista-de-cidadess:
        lista_de_cidades.append(self)
        # cria um dicionário de distância para cada outra cidade
        self.distance_to = {self.name:0.0}
        if distance_to:
            self.distance_to = distance_to

    def calculate_distances(self): 
        '''
        self --> None

        Calcula a distância da cidade para cada uma 
        das outras cidades no lista_de_cidades, 
        colocando valores em um dicionário chamado
        de self.distance_to com nome de cidades e a distância
        ''' 
        for city in lista_de_cidades:
            tmp_dist = self.point_dist(self.x, self.y, city.x, city.y)
            self.distance_to[city.name] = tmp_dist

    # calcula a distância entre dois pontos cartesianos
    def point_dist(self, x1,y1,x2,y2):
        return ((x1-x2)**2 + (y1-y2)**2)**(0.5)


# Route Class
class Route(object):
    """
    

    Armazena uma lista ordenada de todos os objetos de City no global lista_de_cidades.
    Também armazena informações sobre a rota.

    self.route: lista de cidades em lista_de_cidades. Randomicamente misturado em __init__
    self.length: comprimento da rota em float (full loop)

    self.is_valid_route(): Retorna true se conter todas as cidades de lista_de_cidade apenas uma vez
    self.pr_cits_in_rt(): printa todas as cidades da rota
    self.pr_vrb_cits_in_rt: printa todos os pares ordenados
    """
    def __init__(self):
        # inicia um atributo de rota igual a um randomicamente misturado lista_de_cidades
        self.route = sorted(lista_de_cidades, key=lambda *args: random.random())
        ### calcula o comprimento:
        self.recalc_rt_len()

    def recalc_rt_len(self):
        '''
        self --> None

        Método para recalcular o comprimento da rota
        se o atributo self.route foi alterado manualmente.
        '''
        #zera o comprimento
        self.length = 0.0
        # para cada cidade no atributo de rota
        for city in self.route:
            # seta uma próxima variável de cidade que aponta para a próxima cidade na lista
            next_city = self.route[self.route.index(city)-len(self.route)+1]
            # Usa o atributo distance_to da primeira cidade para encontrar a distância até a próxima cidade:
            dist_to_next = city.distance_to[next_city.name]
            self.length += dist_to_next

    def pr_cits_in_rt(self, print_route=False):
        '''
        self --> None

        printa todas as cidades da rota
        '''
        cities_str = ''
        for city in self.route:
            cities_str += city.name + ','
        cities_str = cities_str[:-1] # chops off last comma
        if print_route:
            print('    ' + cities_str)

    def pr_vrb_cits_in_rt(self):
        '''
        self --> None

        printa todos os pares ordenados
        '''
        cities_str = '|'
        for city in self.route:
            cities_str += str(city.x) + ',' + str(city.y) + '|'
        print(cities_str)

    def is_valid_route(self):
        '''
        self --> Bool()

        Retorna true se conter todas as cidades de lista_de_cidade apenas uma vez
        Retorna falso se tiver duplicatas
        '''
        for city in lista_de_cidades:
            if self.count_mult(self.route,lambda c: c.name == city.name) > 1:
                return False
        return True

    
    def count_mult(self, seq, pred):
        return sum(1 for v in seq if pred(v))


# População de rotas
class RoutePop(object):
    """
    Contém uma lista de rotas e informações sobre elas

    self.rt_pop: lista de rotas (object)
    self.size: tamanho de pop definido em init
    self.fittest: rota com menor tamanho de self.rt_pop

    self.get_fittest():  Calcula a rota com maior fitness
    """
    def __init__(self, size, initialise):
        self.rt_pop = []
        self.size = size
        # se quisermos inicializar uma population.rt_pop:
        if initialise:
            for x in range(0,size):
                new_rt = Route()
                self.rt_pop.append(new_rt)
            self.get_fittest()

    def get_fittest(self):
        '''
        self --> Route()

        
        Retorna as duas rotas mais curtas da população, em uma lista chamada self.top_two
        '''
        # classifica a lista com base nos comprimentos das rotas
        sorted_list = sorted(self.rt_pop, key=lambda x: x.length, reverse=False)
        self.fittest = sorted_list[0]
        return self.fittest


# Agrupando métodos de GA
class GA(object):
    """
    Classe para executar o algoritmo genético.

    crossover(parent1, parent2): Retorna uma rota secundária após criar as duas rotas pai.

    """
    def crossover_experimental(routeA,routeB):
        '''
            
            Algoritmo de crossover experimental usando uma idéia de spidering-out. Menos eficaz no momento.
            UMA POSSÍVEL MELHORA AQUI. ISSO PODE SER MUDADO. ATENÇÃO.
        '''
        # novo filho Route()
        child_rt = Route()


        # pr anão precisar recalcular
        routeB_len = len(routeB.route)

        # escolhe uma cidade randomicamente
        random_city = random.choice(lista_de_cidades)

        # routeA pra baixo
        # routeB pra cima

        incrementing_a = True
        incrementing_b = True

        idx_a = routeA.route.index(random_city)
        idx_b = routeB.route.index(random_city)

        idx_a -= 1
        idx_b += 1

        if idx_a < 0:
            incrementing_a = False

        if idx_b >= routeB_len:
            incrementing_b = False

        child_rt.route = [random_city]

        # print(random_city.name)

        while (incrementing_a and incrementing_b):
            # print('idx_a: {}'.format(idx_a))

            if idx_a >= 0:
                if not (routeA.route[idx_a] in child_rt.route):
                    child_rt.route.insert(0, routeA.route[idx_a])

            idx_a -= 1

            if idx_a < 0:
                incrementing_a = False
                break

            # child_rt.pr_cits_in_rt()


            if idx_b < routeB_len:
                if not (routeB.route[idx_b] in child_rt.route):
                    child_rt.route.append(routeB.route[idx_b])

            idx_b += 1

            if idx_b >= routeB_len:
                incrementing_b = False
                break

            # print('idx_b: {}'.format(idx_b))
            # child_rt.pr_cits_in_rt()

        # um dos incrementing tem deve ser falso

        shuffled_cities = sorted(routeA.route, key=lambda *args: random.random())
        for city in shuffled_cities:
            if not city in child_rt.route:
                child_rt.route.append(city)

        return child_rt

    def crossover(self, parent1, parent2):
        '''
        Route(), Route() --> Route()

        Retorna uma rota secundária () depois de reproduzir as duas rotas principais.
        As rotas devem ter o mesmo comprimento.

        A criação é feita selecionando-se um intervalo aleatório de parent1 e colocando-o na rota-filha vazia (no mesmo lugar).
        Os intervalos são preenchidos, sem duplicatas, na ordem em que aparecem no pai2.

        Exemplo:
            parent1: 0123456789
            parent1: 5487961320

            start_pos = 0
            end_pos = 4

            unfilled child: 01234*****
            filled child:   0123458796

            * = None

        '''



        # novo filho Route()
        child_rt = Route()

        for x in range(0,len(child_rt.route)):
            child_rt.route[x] = None

        # Dois índices inteiros aleatórios do pai1:
        start_pos = random.randint(0,len(parent1.route))
        end_pos = random.randint(0,len(parent1.route))


        #### toma a sub-rota do pai um e enfima em si mesmo
        # se a posição inicial é antes do fim:
        if start_pos < end_pos:
            # faça na ordem começo-->fim
            for x in range(start_pos,end_pos):
                child_rt.route[x] = parent1.route[x] 
        # se a posição inicial é depois do fim:
        elif start_pos > end_pos:
            # faça na ordem fim-->começo
            for i in range(end_pos,start_pos):
                child_rt.route[i] = parent1.route[i] 


        # Percorre o pai2. E preenche o child_rt
        # percorre o comprimento do pai2:
        for i in range(len(parent2.route)):
            # se pai2 tiver uma cidade que a criança ainda não tenha:
            if not parent2.route[i] in child_rt.route:
                # bota no primeiro ponto 'vazio' e sai do loop
                for x in range(len(child_rt.route)):
                    if child_rt.route[x] == None:
                        child_rt.route[x] = parent2.route[i]
                        break
        # repetido até que todas as cidades estejam na rota do filho

        # retorna a rota filha (do tipo Route ())
        child_rt.recalc_rt_len()
        return child_rt

    def mutate(self, route_to_mut):
        '''
        Route() --> Route()

        Troca dois índices aleatórios em  route_to_mut.route. Faz isso k_mut_prob*100 % das vezes
        '''
        # k_mut_prob %
        if random.random() < k_mut_prob:

            # dois indices aleatorios:
            mut_pos1 = random.randint(0,len(route_to_mut.route)-1)
            mut_pos2 = random.randint(0,len(route_to_mut.route)-1)

            if mut_pos1 == mut_pos2:
                return route_to_mut

            # se não são iguais faz o swap:
            city1 = route_to_mut.route[mut_pos1]
            city2 = route_to_mut.route[mut_pos2]

            route_to_mut.route[mut_pos2] = city1
            route_to_mut.route[mut_pos1] = city2

        # Recalcula o comprimento da rota (atualiza comprimento)
        route_to_mut.recalc_rt_len()

        return route_to_mut

    def mutate_2opt(route_to_mut):
        '''
        Route() --> Route()

        Troca dois índices aleatórios em  route_to_mut.route. Faz isso k_mut_prob*100 % das vezes
        Versão alternativa
        '''
        # k_mut_prob %
        if random.random() < k_mut_prob:

            for i in range(len(route_to_mut.route)):
                for ii in range(len(route_to_mut.route)): # i is a, i + 1 is b, ii is c, ii+1 is d
                    if (route_to_mut.route[i].distance_to[route_to_mut.route[i-len(route_to_mut.route)+1].name]
                     + route_to_mut.route[ii].distance_to[route_to_mut.route[ii-len(route_to_mut.route)+1].name]
                     > route_to_mut.route[i].distance_to[route_to_mut.route[ii].name]
                     + route_to_mut.route[i-len(route_to_mut.route)+1].distance_to[route_to_mut.route[ii-len(route_to_mut.route)+1].name]):

                        c_to_swap = route_to_mut.route[ii]
                        b_to_swap = route_to_mut.route[i-len(route_to_mut.route)+1]

                        route_to_mut.route[i-len(route_to_mut.route)+1] = c_to_swap
                        route_to_mut.route[ii] = b_to_swap 

            route_to_mut.recalc_rt_len()

        return route_to_mut

    def tournament_select(self, population):
        '''
        RoutePop() --> Route()

        

        Seleciona aleatoriamente a quantia de tournament_size de Routes () da população de entrada.
        Leva o mais forte do menor número de rotas (). 
        
        Aqui vale um disclaimer: torunament_size é tamanho maximo de rotas permitidas pra disputa.. 
        é um jargão comum nos algoritmos de ga

        Principio: da uma chance para rotas ruins seguirem em frente mas favorece as rotas melhores.
        '''

        #nova população não inicializada
        tournament_pop = RoutePop(size=tournament_size,initialise=False)

        # prrenchida com novos individuos randons
        for i in range(tournament_size-1):
            tournament_pop.rt_pop.append(random.choice(population.rt_pop))
        
        # retorna o que tem maior fitness:
        return tournament_pop.get_fittest()

    def evolve_population(self, init_pop):
        '''
        RoutePop() --> RoutePop()

        Pega uma população e evolui, em seguida, retorna a nova população.
        '''

        #faz uma nova população:
        descendant_pop = RoutePop(size=init_pop.size, initialise=True)

        # meritocracia offset (quantidade de rotas () transportadas para nova população)
        meritocraciaOffset = 0

        # se tivermos meritocracia, definiremos o primeiro da nova população para o mais forte dos velhos
        if meritocracia:
            descendant_pop.rt_pop[0] = init_pop.fittest
            meritocraciaOffset = 1

        # Atravessa a nova população e a preenche com o filho de dois vencedores de torneios da população anterior
        for x in range(meritocraciaOffset,descendant_pop.size):
            # dois pais:
            tournament_parent1 = self.tournament_select(init_pop)
            tournament_parent2 = self.tournament_select(init_pop)

            # filho:
            tournament_child = self.crossover(tournament_parent1, tournament_parent2)

            # preenche com os filhos
            descendant_pop.rt_pop[x] = tournament_child

        # Muta todas as rotas (a mutação acontece com um prob p = k_mut_prob)
        for route in descendant_pop.rt_pop:
            if random.random() < 0.3:
                self.mutate(route)

        # atualiza a rota com mais fitness:
        descendant_pop.get_fittest()

        return descendant_pop



# Classe para fazer o acompanhamento visual. Não criei nem mexi em nada do que tá abaixo.


class App(object):
    """
    Runs the application
    """
    def __init__(self,n_generations,pop_size, graph=False):
        '''
        Initiates an App object to run for n_generations with a population of size pop_size
        '''

        if csv_cities:
            self.read_csv()

        self.n_generations = n_generations
        self.pop_size = pop_size

        # Once all the cities are defined, calcualtes the distances for all of them.
        # for city in lista_de_cidades:
        #     city.calculate_distances()
        if graph:
            self.set_city_gcoords()
            
            # Initiates a window object & sets its title
            self.window = Tk()
            self.window.wm_title("Generation 0")

            # initiates two canvases, one for current and one for best
            self.canvas_current = Canvas(self.window, height=300, width=300)
            self.canvas_best = Canvas(self.window, height=300, width=300)

            # Initiates two labels
            self.canvas_current_title = Label(self.window, text="Best route of current gen:")
            self.canvas_best_title = Label(self.window, text="Overall best so far:")

            # Initiates a status bar with a string
            self.stat_tk_txt = StringVar()
            self.status_label = Label(self.window, textvariable=self.stat_tk_txt, relief=SUNKEN, anchor=W)

            # creates dots for the cities on both of the canvases
            for city in lista_de_cidades:
                self.canvas_current.create_oval(city.graph_x-2, city.graph_y-2, city.graph_x + 2, city.graph_y + 2, fill='blue')
                self.canvas_best.create_oval(city.graph_x-2, city.graph_y-2, city.graph_x + 2, city.graph_y + 2, fill='blue')

            # Packs all the widgets (physically creates them and places them in order)
            self.canvas_current_title.pack()
            self.canvas_current.pack()
            self.canvas_best_title.pack()
            self.canvas_best.pack()
            self.status_label.pack(side=BOTTOM, fill=X)

            # Runs the main window loop
            self.window_loop(graph)
        else:
            print("Calculating GA_loop")
            self.GA_loop(n_generations,pop_size, graph=graph)

    def set_city_gcoords(self):
        '''
        All cities have graph_x and graph_y attributes that are only referenced when showing them on the map.
        This method takes the original city.x and city.y and transforms them so that the coordinates map fully onto the 300x300 map view.
        '''

        # defines some variables (we will set them next)
        min_x = 100000
        max_x = -100000
        min_y = 100000
        max_y = -100000

        #finds the proper maximum/minimum
        for city in lista_de_cidades:

            if city.x < min_x:
                min_x = city.x
            if city.x > max_x:
                max_x = city.x

            if city.y < min_y:
                min_y = city.y
            if city.y > max_y:
                max_y = city.y

        # shifts the graph_x so the leftmost city starts at x=0, same for y.
        for city in lista_de_cidades:
            city.graph_x = (city.graph_x + (-1*min_x))
            city.graph_y = (city.graph_y + (-1*min_y))

        # resets the variables now we've made changes
        min_x = 100000
        max_x = -100000
        min_y = 100000
        max_y = -100000

        #finds the proper maximum/minimum
        for city in lista_de_cidades:

            if city.graph_x < min_x:
                min_x = city.graph_x
            if city.graph_x > max_x:
                max_x = city.graph_x

            if city.graph_y < min_y:
                min_y = city.graph_y
            if city.graph_y > max_y:
                max_y = city.graph_y

        # if x is the longer dimension, set the stretch factor to 300 (px) / max_x. Else do it for y. This conserves aspect ratio.
        if max_x > max_y:
            stretch = 300 / max_x
        else:
            stretch = 300 / max_y

        # stretch all the cities so that the city with the highest coordinates has both x and y < 300
        for city in lista_de_cidades:
            city.graph_x *= stretch
            city.graph_y = 300 - (city.graph_y * stretch)


    def update_canvas(self,the_canvas,the_route,color):
        '''
        Convenience method to update the canvases with the new routes
        '''

        # deletes all current items with tag 'path'
        the_canvas.delete('path')

        # loops through the route
        for i in range(len(the_route.route)):

            # similar to i+1 but will loop around at the end
            next_i = i-len(the_route.route)+1

            # creates the line from city to city
            the_canvas.create_line(the_route.route[i].graph_x,
                                the_route.route[i].graph_y,
                                the_route.route[next_i].graph_x,
                                the_route.route[next_i].graph_y,
                                tags=("path"),
                                fill=color)

            # Packs and updates the canvas
            the_canvas.pack()
            the_canvas.update_idletasks()

    def read_csv(self):
        with open(csv_name, 'rt') as f:
            reader = csv.reader(f)
            for row in reader:
                new_city = City(row[0],float(row[1]),float(row[2]))

    def GA_loop(self,n_generations,pop_size, graph=False):
        '''
        Main logic loop for the GA. Creates and manages populations, running variables etc.
        '''

        # takes the time to measure the elapsed time
        start_time = time.time()

        # Creates the population:
        print("Creates the population:")
        the_population = RoutePop(pop_size, True)
        print ("Finished Creation of the population")

        # the_population.rt_pop[0].route = [1,8,38,31,44,18,7,28,6,37,19,27,17,43,30,36,46,33,20,47,21,32,39,48,5,42,24,10,45,35,4,26,2,29,34,41,16,22,3,23,14,25,13,11,12,15,40,9]
        # the_population.rt_pop[0].recalc_rt_len()
        # the_population.get_fittest()

        #checks to make sure there are no duplicate cities:
        if the_population.fittest.is_valid_route() == False:
            raise NameError('Multiple cities with same name. Check cities.')
            return # if there are, raise a NameError and return

        # gets the best length from the first population (no practical use, just out of interest to see improvements)
        initial_length = the_population.fittest.length

        # Creates a random route called best_route. It will store our overall best route.
        best_route = Route()

        if graph:
            # Update the two canvases with the just-created routes:
            self.update_canvas(self.canvas_current,the_population.fittest,'red')
            self.update_canvas(self.canvas_best,best_route,'green')


        # Main process loop (for number of generations)
        for x in range(1,n_generations):
            # Updates the current canvas every n generations (to avoid it lagging out, increase n)
            if x % 8 == 0 and graph:
                self.update_canvas(self.canvas_current,the_population.fittest,'red')

            # Evolves the population:
            the_population = GA().evolve_population(the_population)

            # If we have found a new shorter route, save it to best_route
            if the_population.fittest.length < best_route.length:
                # set the route (copy.deepcopy because the_population.fittest is persistent in this loop so will cause reference bugs)
                best_route = copy.deepcopy(the_population.fittest)
                if graph:
                    # Update the second canvas because we have a new best route:
                    self.update_canvas(self.canvas_best,best_route,'green')
                    # update the status bar (bottom bar)
                    self.stat_tk_txt.set('Initial length {0:.2f} Best length = {1:.2f}'.format(initial_length,best_route.length))
                    self.status_label.pack()
                    self.status_label.update_idletasks()

            # Prints info to the terminal:
            self.clear_term()
            print('Generation {0} of {1}'.format(x,n_generations))
            print(' ')
            print('Overall fittest has length {0:.2f}'.format(best_route.length))
            print('and goes via:')
            best_route.pr_cits_in_rt(True)
            print(' ')
            print('Current fittest has length {0:.2f}'.format(the_population.fittest.length))
            print('And goes via:')
            the_population.fittest.pr_cits_in_rt(True)
            print(' ')
            print('''The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.''')

            if graph:
                # sets the window title to the latest Generation:
                self.window.wm_title("Generation {0}".format(x))
        if graph:
            # sets the window title to the last generation
            self.window.wm_title("Generation {0}".format(n_generations))

            # updates the best route canvas for the last time:
            self.update_canvas(self.canvas_best,best_route,'green')
            
        # takes the end time of the run:
        end_time = time.time()

        # Prints final output to terminal:
        self.clear_term()
        print('Finished evolving {0} generations.'.format(n_generations))
        print("Elapsed time was {0:.1f} seconds.".format(end_time - start_time))
        print(' ')
        print('Initial best distance: {0:.2f}'.format(initial_length))
        print('Final best distance:   {0:.2f}'.format(best_route.length))
        print('The best route went via:')
        best_route.pr_cits_in_rt(print_route=True)

    def window_loop(self, graph):
        '''
        Wraps the GA_loop() method and initiates the window on top of the logic.
        window.mainloop() hogs the Thread, that's why the GA_loop is called as a callback
        '''
        # see http://stackoverflow.com/questions/459083/how-do-you-run-your-own-code-alongside-tkinters-event-loop
        self.window.after(0,self.GA_loop(self.n_generations, self.pop_size, graph))
        self.window.mainloop()

    # Helper function for clearing terminal window
    def clear_term(self):
        os.system('cls' if os.name=='nt' else 'clear')


def random_cities():
    PortoAlegre = City('PortoAlegre', 245, 135)
    Caxias = City('Caxias', 250, 182)
    Pelotas = City('Pelotas', 193, 38)
    SantaMaria = City('SantaMaria', 125, 150)
    RioGrande = City('RioGrande', 208, 25)
    PassoFundo = City('PassoFundo', 184, 231)
    SantaCruz = City('SantaCruz', 183, 150)
    Uruguaiana = City('Uruguaiana', -25, 145)
    Itaqui = City('Itaqui', -5, 170)
    SaoBorja = City('SaoBorja', 10, 210)
    Bage = City('Bage', 110, 65)
    Santiago= City('Santiago', 72, 184)
    CruzAlta = City('CruzAlta', 136, 215)
    Ijui = City('Ijui', 118, 225)
    Santana = City('Santana', 43, 85)
    SantoAngelo = City('SantoAngelo', 95, 233)
    Alegrete = City('Alegrete', 30, 150)
    Rosario = City('Rosario', 70, 125)
    SaoGabriel = City('SaoGabriel', 104, 118)

    for city in lista_de_cidades:
        city.calculate_distances()
    ######## create and run an application instance:
    app = App(n_generations=k_n_generations,pop_size=k_population_size, graph=True)

if __name__ == '__main__':
    random_cities()



Creates the population:
Finished Creation of the population
Generation 1 of 100
 
Overall fittest has length 1603.97
and goes via:
    SantoAngelo,SantaMaria,SaoGabriel,RioGrande,Pelotas,Bage,SantaCruz,Alegrete,Itaqui,SaoBorja,Rosario,Santana,Uruguaiana,Ijui,CruzAlta,Caxias,PortoAlegre,PassoFundo,Santiago
 
Current fittest has length 1603.97
And goes via:
    SantoAngelo,SantaMaria,SaoGabriel,RioGrande,Pelotas,Bage,SantaCruz,Alegrete,Itaqui,SaoBorja,Rosario,Santana,Uruguaiana,Ijui,CruzAlta,Caxias,PortoAlegre,PassoFundo,Santiago
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 2 of 100
 
Overall fittest has length 1509.97
and goes via:
    Alegrete,Rosario,SaoGabriel,RioGrande,Pelotas,PortoAlegre,SantaCruz,Bage,Santana,Uruguaiana,CruzAlta,PassoFundo,Ijui,SantoAngelo,Caxias,SantaMaria,Santiago,SaoBorja,Itaqui
 
Current fittest has length 1509.97
And goes via:
    Alegrete,Rosario,SaoGabriel,RioGrande,Pelotas,Po

Generation 15 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 16 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P

Generation 30 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 31 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P

Generation 44 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 45 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P

Generation 59 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 60 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P

Generation 73 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 74 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P

Generation 87 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
The screen with the maps may become unresponsive if the population size is too large. It will refresh at the end.
Generation 88 of 100
 
Overall fittest has length 1039.64
and goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,PortoAlegre,RioGrande,Pelotas,SantaCruz,SantaMaria,SaoGabriel,Bage
 
Current fittest has length 1039.64
And goes via:
    Santana,Rosario,Alegrete,Uruguaiana,Itaqui,SaoBorja,Santiago,SantoAngelo,Ijui,CruzAlta,PassoFundo,Caxias,P